# Monte Carlo Simulation for FE
## IEOR 4703

### Pricing Black-Merton-Scholes
#### Utilizing various discretization schemes

In [3]:
import numpy as np
# import our BMS price function
from BMS import BMS_price
from time import time

### Option parameters:

In [4]:
s0 = 100
K = 100
T = 1
sig = 0.35
r = 0.10
q = 0.0125

### True price:

In [5]:
c = BMS_price('call', s0, K, r, q, sig, T)
print('c = %f' % c)

c = 17.685940


### Discretization of SDE using 3 schemes: Euler, Milstein, and Runge-Kutta

In [6]:
n_sims = 50000
m = 52
dt = 1. * T / m

In [7]:
np.random.seed(1167934)

In [8]:
st = time()

payoff1 = 0
payoff2 = 0
payoff3 = 0
payoff4 = 0

for j in range(n_sims):

    s1 = s0
    s2 = s0
    s3 = s0

    z = np.random.randn(m)
    for i in range(m):

        # Euler
        s1 = s1 + (r - q)*s1*dt + sig*s1*np.sqrt(dt)*z[i]

        # Milstein
        s2 = s2 + (r - q)*s2*dt + sig*s2*np.sqrt(dt)*z[i] + 0.5*sig*sig*s2*dt*(z[i]**2 - 1)

        # Runge-Kutta
        s3_tilde = s3 + (r - q)*s3*dt + sig*s3*np.sqrt(dt)
        s3 = s3 + (r - q)*s3*dt + sig*s3*np.sqrt(dt)*z[i] + 0.5*(sig*s3_tilde - sig*s3)*(z[i]**2-1)*np.sqrt(dt)

    # exact simulation (Ito solution)
    s4 = s0*np.exp((r - q - sig**2/2)*T + sig*np.sqrt(T)*z[-1])

    payoff1 += max(s1 - K, 0)
    payoff2 += max(s2 - K, 0)
    payoff3 += max(s3 - K, 0)
    payoff4 += max(s4 - K, 0)

c1 = np.exp(-r*T) * payoff1 / n_sims
c2 = np.exp(-r*T) * payoff2 / n_sims
c3 = np.exp(-r*T) * payoff3 / n_sims
c4 = np.exp(-r*T) * payoff4 / n_sims

et = time()
print('Elapsed times was %f seconds' % (et-st))

Elapsed times was 22.496684 seconds


In [9]:
print('exact solution        = %f' % c)
print('Euler                 = %f' % c1)
print('Milstein              = %f' % c2)
print('Runge-Kutta           = %f' % c3)
print('MC w/o discretization = %f' % c4)

exact solution        = 17.685940
Euler                 = 17.992613
Milstein              = 17.966577
Runge-Kutta           = 17.965961
MC w/o discretization = 17.756525
